<a href="https://colab.research.google.com/github/nywkim/project/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

sp = pd.read_csv('data_ind.csv', usecols=['acousticness', 'danceability', 'duration_ms', 'energy', 'liveness', 'loudness', 'speechiness',
                'tempo', 'valence', 'song_title', 'artist', 'target', 'key'])
sp.columns = ['Acousticness', 'Danceability', 'Duration (ms)', 'Energy', 'Chord',
       'Liveness', 'Loudness', 'Speechiness', 'Tempo', 'Valence', 'target', 'Song Name', 'Artist']
test = pd.read_csv('spotify_dataset.csv', usecols=['Acousticness', 'Danceability', 'Duration (ms)', 'Energy',
       'Liveness', 'Loudness', 'Speechiness', 'Tempo', 'Valence', 'Song Name', 'Artist', 'Chord'])
test['Chord'] = test.Chord.replace(['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B'], [0,1,2,3,4,5,6,7,8,9,10,11])


In [ ]:
import plotly.express as px

sp.drop_duplicates(subset=None,inplace=True)

target = 'target'
sp_t = sp[target].value_counts()
px.pie(sp, values=sp_t, names=['good','bad'], title="Liked/Unliked Songs Distribution Pie Chart", 
       color_discrete_sequence=["#1DB954", "#D22B2B"])

In [ ]:
sp_s = sp.sort_index(axis=1)
test_s = test.sort_index(axis=1)

sp = sp_s.drop(columns=['Song Name', 'Artist', 'target'], axis=1)
X_test = test_s.drop(columns=['Song Name', 'Artist'], axis=1)
sp_ = sp_s[['target','Song Name', 'Artist']]


In [ ]:
sp = pd.concat([sp,sp_],axis = 1)
train, val = train_test_split(sp, test_size=0.2, random_state=17)
features = sp.drop(columns=[target]).columns
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import tree, ensemble
from xgboost import XGBClassifier
!pip install catboost
from catboost import CatBoostClassifier
import xgboost as xgboost

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score


In [ ]:
obj=list(np.where(X_train.dtypes == np.object)[0])
obj

In [ ]:
model=CatBoostClassifier()
model.fit(X_train,y_train,cat_features=obj)
print(classification_report(y_val,model.predict(X_val)))

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def vif_show(X_vif):
    vif = pd.DataFrame()
    vif['Features'] = X_vif.columns
    vif['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    print(vif)
    print()
    if(vif.iloc[0,1] > 10.0 ):
        col = vif.iloc[0,0]
        X_vif.drop([vif.iloc[0,0]],axis =1, inplace = True)
        print("After removing \""+ col + "\" from datafame")
        vif_show(X_vif)

vif_show(sp)